### 5th IAF mean

In [1]:
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
import os
from glob import glob
from collections import OrderedDict
import xarray as xr
import numpy as np
import esmlab
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import cartopy
import cartopy.crs as ccrs
import seawater as sw
from seawater.library import T90conv
from scipy import stats
import pop_tools 

from pint import UnitRegistry
import ncar_jobqueue
from dask.distributed import Client

units = UnitRegistry()
units.define('micromoles_per_kilogram = umol / kg') # used in WOA datasets

Cannot write to data cache folder '/glade/p/cesmdata/cseg'. Will not be able to download remote data files. Use environment variable 'CESMDATAROOT' to specify another directory.


In [2]:
cluster = ncar_jobqueue.NCARCluster(project='ncgd0011')
cluster.adapt(minimum_jobs=0, maximum_jobs=30)
client = Client(cluster)
client

Client Scheduler: tcp://10.12.205.14:43590 Dashboard: https://jupyterhub.ucar.edu/dav/user/kristenk/proxy/8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


### MONTHLY variables

#### Define CESM case and variable

In [27]:
case = 'g.e22b05.G1850ECOIAF_JRA.TL319_g17.cocco.001'
user = 'kristenk'
path = '/glade/scratch/' + user + '/archive/' + case + '/ocn/proc/tseries/month_1'

#### cocco run

In [28]:
variables = ['sp_Fe_lim_surf','sp_P_lim_surf','sp_N_lim_surf',
             'diat_Fe_lim_surf', 'diat_P_lim_surf','diat_N_lim_surf','diat_SiO3_lim_surf','diaz_P_lim_surf',
             'diaz_Fe_lim_surf','photoC_TOT_zint','photoC_sp_zint','photoC_diat_zint','photoC_diaz_zint','photoC_cocco_zint',
             'cocco_Fe_lim_surf','cocco_C_lim_surf','cocco_N_lim_surf','cocco_P_lim_surf',
             'CaCO3_PROD_zint','SiO2_PROD','POC_FLUX_100m','coccoCaCO3','diatC', 'coccoC','spC','diazC',
             'PO4','Fe','NO3','SiO3','ALK','O2'] 

#### 3 PFT run

In [29]:
# #variables = ['sp_Fe_lim_surf','sp_P_lim_surf','sp_N_lim_surf',
#              'diat_Fe_lim_surf', 'diat_P_lim_surf','diat_N_lim_surf','diat_SiO3_lim_surf',
#              'diaz_P_lim_surf','diaz_Fe_lim_surf','photoC_TOT_zint','photoC_sp_zint',
#              'photoC_diat_zint','photoC_diaz_zint',
#              'CaCO3_PROD_zint','SiO2_PROD','POC_FLUX_100m','spCaCO3','diatC', 
#              'spC','diazC','ALK','SALT'] 
#variables = ['PO4','Fe','NO3','SiO3'] 
#variables = ['O2']

In [30]:
# IAFm = '000101-006112'
# IAF = 'IAF1'

IAFm='006201-012212'
IAF='IAF2'

#IAFm='012301-018312'
#IAF='IAF3'

# IAFm='018401-024412'
# IAF='IAF4'

#### Get last IAF cycle

In [33]:
%%time
ds_list = []
print(f'reading {case}')
ds = xr.Dataset()

for v in variables:
    
    print(v)
    
    files = sorted(glob(f'{path}/{case}.pop.h.{v}.{IAFm}.nc'))       
    
    dsv=xr.open_mfdataset(files, data_vars="minimal", coords='minimal', compat="override", parallel=True, concat_dim="time",
                       drop_variables=["transport_components", "transport_regions"], decode_times=True, chunks={'z_t' : 1})

    ds = xr.merge((ds, dsv))
    ds_list.append(ds)

reading g.e22b05.G1850ECOIAF_JRA.TL319_g17.cocco.001
sp_Fe_lim_surf
sp_P_lim_surf
sp_N_lim_surf
diat_Fe_lim_surf
diat_P_lim_surf
diat_N_lim_surf
diat_SiO3_lim_surf
diaz_P_lim_surf
diaz_Fe_lim_surf
photoC_TOT_zint
photoC_sp_zint
photoC_diat_zint
photoC_diaz_zint
photoC_cocco_zint
cocco_Fe_lim_surf
cocco_C_lim_surf
cocco_N_lim_surf
cocco_P_lim_surf
CaCO3_PROD_zint
SiO2_PROD
POC_FLUX_100m
coccoCaCO3
diatC
coccoC
spC
diazC
SALT
PO4
Fe
NO3
SiO3
ALK
O2
CPU times: user 52.5 s, sys: 3.35 s, total: 55.8 s
Wall time: 3min 5s


In [34]:
coords = {'x':'TLONG','y':'TLAT'}
keepthese=['z_t','time_bound','TAREA','REGION_MASK'] + variables
keep_vars = keepthese +list(coords.values())+['dz','KMT']
ds = ds.drop([v for v in ds.variables if v not in keep_vars])

In [35]:
%%time 
cesm_ann_ds=ds.mean(dim='time').compute()

distributed.scheduler - ERROR - Couldn't gather keys {"('mean_agg-aggregate-48e8d6dcbeff2bc8cced785d0f9ba15f', 26, 0, 0)": [], "('mean_agg-aggregate-48e8d6dcbeff2bc8cced785d0f9ba15f', 12, 0, 0)": []} state: ['waiting', 'waiting'] workers: []
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: [], ('mean_agg-aggregate-48e8d6dcbeff2bc8cced785d0f9ba15f', 26, 0, 0)
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: [], ('mean_agg-aggregate-48e8d6dcbeff2bc8cced785d0f9ba15f', 12, 0, 0)
NoneType: None


CPU times: user 57.2 s, sys: 11.6 s, total: 1min 8s
Wall time: 35min 25s


In [36]:
for v in variables:
    
    keep_vars = ['z_t','time_bound','TAREA','time','dz','KMT', v]  + list(coords.values())
    
    ds_out = cesm_ann_ds.drop([v for v in cesm_ann_ds.variables if v not in keep_vars])
    
    outfile='/glade/scratch/kristenk/archive/'+case+'/ocn/proc/mean'+IAF+'/mean_'+IAF+'_'+v+'.nc'
    ds_out.to_netcdf(outfile)

### ANNUAL variables

In [43]:
case = 'cesm22cocco.G1850ECOIAF.T62_g17.010'
#path = '/glade/scratch/kristenk/archive/' + case + '/ocn/proc/tseries/year_1'
path = '/glade/campaign/cgd/oce/projects/besome/cesm-cases/' + case + '/ocn/proc/tseries/year_1'
variables = ['CaCO3_ALT_CO2_FLUX_IN']

In [44]:
%%time

ds_list = []

print(f'reading {case}')

ds = xr.Dataset()

for v in variables:
    
    print(v)
    
    files = sorted(glob(f'{path}/{case}.pop.h.ecosys.nyear1.{v}.0249-0310.nc'))

    dsv=xr.open_mfdataset(files, data_vars="minimal", coords='minimal', compat="override", parallel=True, concat_dim="time",
                       drop_variables=["transport_components", "transport_regions"], decode_times=True)

    
    ds = xr.merge((ds, dsv))
    ds_list.append(ds)

reading cesm22cocco.G1850ECOIAF.T62_g17.010
CaCO3_ALT_CO2_FLUX_IN
CPU times: user 463 ms, sys: 109 ms, total: 572 ms
Wall time: 31.8 s


In [45]:
coords = {'x':'TLONG','y':'TLAT'}
keepthese=['z_t','z_t_150m','time_bound','TAREA','REGION_MASK'] + variables
keep_vars = keepthese +list(coords.values())+['dz','KMT']
ds = ds.drop([v for v in ds.variables if v not in keep_vars])

In [46]:
ds=ds.mean(dim='time')

In [47]:
for v in variables:
        outfile='/glade/scratch/kristenk/archive/'+case+'/ocn/proc/mean5thIAF/mean_IAF5_'+v+'.nc'
        ds[v].to_netcdf(outfile)